In [1]:
import sys
sys.path.append('/Users/russojd/Research/synthMD/synthetic_md')

In [2]:
import synthetic_md.models as smd_models
import tqdm.auto as tqdm
import MDAnalysis as mda
import numpy as np

## Load saved transition matrix / structures

To run synthetic MD with a generating MSM, we need:
- A continuous trajectory
- A corresponding discrete trajectory
- A transition matrix, built from those trajectories.

The transition matrix is used to generate discrete synthetic MD trajectories.
The discrete and continuous trajectories are used to map discrete states to continuous structures (which allows us to convert a discrete generated trajectory into a full-coordinate MD trajectory).

**Replace the contents of these cells with however you store those.**

In [4]:
continuous_trajectory = mda.Universe(
    '/Users/russojd/Research/desres_trajectories/DESRES-Trajectory_2JOF-0-protein 2/2JOF-0-protein/2JOF.pdb',
    '/Users/russojd/Research/protein-like_msm/aligned_2JOF.dcd')

stored_msm = np.load('/Users/russojd/Research/protein-like_msm/strict_strat_tic_cluster_lag-1ns_transition_matrix_ernesto_symm.npz', allow_pickle=True)
discrete_trajectory = stored_msm['discrete_trajectory']
fine_transition_matrix = stored_msm['transition_matrix']

n_states = len(fine_transition_matrix)

This just parameterizes the output of the SynMD.

In [42]:
# Output data will be put in this directory. This seed is used to seed the synMD RNG, so re-running this notebook with the same seed will produce the same trajectories.

seed = 11
gen_path = f'/Users/russojd/Research/synthMD/experiments/generated/2j0f-{seed}-1ns'

# The number of SynMD trajectories to produce, and their length.
n_trajectories = 800
trajectory_length = 500

## Monkey-patch generative model class

Before building the generative model, we need to define how it'll do some things.

Define how the generative model will choose structures, given a state index.

This function defines how to backmap a discrete state index to a representative full-coordinate structure, to be used in backmapping the discrete trajectories to a full-coordinate trajectory.

Here, I show **two example ways** you could do this.

The first way: Here, we randomly choose structures from any points in the continuous trajectory that match this cluster.

You could have a bunch of different structures mapped to a single cluster, and this will randomly pick from them.

In [6]:
def assign_structure(self, state_index):
    """
    Given a state index, return a representative structure.

    :param self:
    :param state_index:
    :return:
    """

    valid_structures_idxs = np.argwhere(discrete_trajectory == state_index).flatten().astype(int)

    assert len(valid_structures_idxs) > 0, f"No valid structures found for state {state_index}"

    structure_idx = self.rng.choice(valid_structures_idxs)
    structure = continuous_trajectory.trajectory[structure_idx].positions

    return structure

smd_models.GenerativeMarkovModel.assign_structure = assign_structure

A second (much faster) way: Just map single structures to each discrete state.

In [5]:
# You can uncomment this, and the line in the cell below to associate an RMSD with each discrete state.
#   Just a convenience thing for later analysis.

# rmsd_trajectory = np.load(f'/Users/russojd/Research/protein-like_msm/rmsd_2JOF.npz')['rmsd']

In [ ]:
structure_map = {}
rng = np.random.default_rng(seed=seed)

for state_index in range(n_states):

    valid_structures_idxs = np.argwhere(discrete_trajectory == state_index).flatten().astype(int)
    assert len(valid_structures_idxs) > 0, f"No valid structures found for state {state_index}"
    structure_idx = rng.choice(valid_structures_idxs)

    structure_map[state_index] = [continuous_trajectory.trajectory[structure_idx].positions,
                                  # rmsd_trajectory[structure_idx],
                                  structure_idx]

def assign_structure(self, state_index):
    """
    Given a state index, return a representative structure.

    :param self:
    :param state_index:
    :return:
    """

    return structure_map[state_index][0]

smd_models.GenerativeMarkovModel.assign_structure = assign_structure

You can do whatever you like for the assignment/backmapping -- for example, say you want some variation in your final generated trajectory. You could do something like load the structure from the dictionary, then permute the coordinates in some way.

Each time a discrete state is assigned to a structure, this function runs, so a single discrete state could map to a variety of slightly-permuted structures.

Now, define how to write a full-coordinate trajectory to an MD file.

(This could probably just be standardized into the `smd_models.GenerativeMarkovModel`)

In [8]:
def write_trajectory(self, structure_file, out_file, coordinate_trajectories):
    """
    Write a full-coordinate trajectory, represented as XYZ coordinates, to an MD file format.

    :param structure_file: File defining the MD structure. Must be usable in an `MDAnalysis.Universe`
    :param out_file: File to write atomistic trajectory to.
    :param coordinate_trajectories: Full-coordinate trajectories.
    :return:
    """

    md_trajectories = []

    for i, coordinate_trajectory in enumerate(tqdm.tqdm(coordinate_trajectories, desc="Writing trajectories ")):

        _out_filename = out_file.split('.')[0] + f'_{i}.' + out_file.split('.')[1]

        md_trajectory = mda.Universe(structure_file)
        md_trajectory.load_new(coordinate_trajectory)

        with mda.Writer(_out_filename, len(md_trajectory.atoms)) as W:

            for frame in md_trajectory.trajectory:
                W.write(md_trajectory)

        md_trajectories.append(md_trajectory)

    return md_trajectories

smd_models.GenerativeMarkovModel.write_trajectory = write_trajectory

Finally, construct the model object.

In [43]:
model = smd_models.GenerativeMarkovModel(transition_matrix=fine_transition_matrix, seed=seed)

## Construct trajectories
Generate discrete trajectories, then backmap them to full-coordinate representations.

First, choose some initial (discrete) states

In [10]:
rng = np.random.default_rng(seed=seed)
initial_states = rng.choice(range(n_states), size=n_trajectories, replace=True)

initial_states = [discrete_trajectory[-1] for x in range(n_trajectories)]

Now, generate the discrete trajectories.

In [23]:
dtrajs = model.generate_discrete_trajectories(n_steps=trajectory_length,
                                              initial_states=initial_states,
                                              n_trajectories=n_trajectories)

50

(Optional) Explicitly remove any trajectories that sample both basins

In [ ]:

for i, traj in enumerate(dtrajs):

        new_dtraj = traj
        while np.isin(stored_msm['states_unfolded'], new_dtraj).any() & \
            np.isin(stored_msm['states_folded'],  new_dtraj).any():

            print(f"Traj {i} contains both folded and unfolded. Generating new...")

            new_dtraj = model.generate_discrete_trajectories(n_steps=trajectory_length,
                                                  initial_states=[initial_states[i]],
                                                  n_trajectories=1)

        dtrajs[i] = new_dtraj

Finally, backmap the discrete trajectories to full-coordinate trajectories.

In [ ]:
continuous_trajectories = model.backmap(dtrajs)

## Write trajectories

First, write the full-coordinate trajectories.
This takes us from arrays of XYZ positions at each frame, to an actual MD file.

This is a bonafide MD file -- you can load this up in VMD and visualize your synMD trajectory.

In [47]:
# Topology corresponding to the generated structures
md_topology_file = '/Users/russojd/Research/desres_trajectories/DESRES-Trajectory_2JOF-0-protein 2/2JOF-0-protein/2JOF.pdb'

md_trajs = model.write_trajectory(
    structure_file=md_topology_file,
    out_file=f'{gen_path}/coordinate_trajectories/generated_full-coords.xtc',
    coordinate_trajectories=continuous_trajectories,
)

Writing trajectories :   0%|          | 0/800 [00:00<?, ?it/s]

In [48]:
np.savez(f'{gen_path}/discrete_trajectories', dtrajs=dtrajs)

In [49]:
np.savez(f'{gen_path}/structure_map', structure_map=structure_map)